<h1> Análisis Avanzado de Partidas – Torneo de Grandes Maestros</h1>
<p> Realizaremos un estudio de ocho grandes maestros de reconocido prestigio internacional tomando todas sus partidas durante los seis últimos meses. La información recopilada en estos cuadernillos puede ser útil para cualquiera de estos jugadores, pues mostrará aspectos y tendencias en el juego que pueden servir para enfocar la preparación de las partidas y también para descubrir debilidades propias.</p>

<h3> <strong>Primera parte:</strong> Selección y análisis de partidas</h3>
<table>
  <tr>
    <td colspan="2" rowspan="2" style="vertical-align: top;">
      <img src="./fotos/Esquema_1.png" width="450"/>
    </td>
    <td align="center">
      <img src="./fotos/1_Carlsen.jpeg" width="100"/><br/>
      Carlsen, M
    </td>
    <td align="center">
      <img src="./fotos/2_Nakamura.jpeg" width="100"/><br/>
      Nakamura, Hi
    </td>
    <td align="center">
      <img src="./fotos/3_Bortnyk.jpeg" width="100"/><br/>
      Bortnyk, O
    </td>
    <td align="center">
      <img src="./fotos/4_Bluebaum.png" width="100"/><br/>
      Bluebaum, M
    </td>
  </tr>
  <tr>
    <td align="center">
      <img src="./fotos/5_Zhigalko.jpeg" width="100"/><br/>
      Zhigalko, S
    </td>
    <td align="center">
      <img src="./fotos/6_Vlassov.jpg" width="100"/><br/>
      Vlassov, N
    </td>
    <td align="center">
      <img src="./fotos/7_Andreikin.png" width="100"/><br/>
      Andreikin, D
    </td>
    <td align="center">
      <img src="./fotos/8_Rustemov.jpeg" width="100"/><br/>
      Rustemov, A
    </td>
  </tr>
</table>

## 0. Carga de librerías necesarias

In [1]:
# Ponerlo en spark-client
#!pip install tqdm
#!pip install pyarrow
#!pip install stockfish

In [2]:
# Módulos estándar de Python
import os
import re
import io
import subprocess
import datetime
import statistics
from collections import defaultdict
from pathlib import Path

#  Librerías externas
import pandas as pd
import numpy as np
from tqdm import tqdm
from stockfish import Stockfish
from pyarrow import fs

#  Módulo específico de ajedrez
import chess.pgn

## 1. Selección de jugadores y del horizonte temporal

In [2]:
jugadores_objetivo = {"Carlsen,M", "Nakamura,Hi", "Bortnyk,Olexandr", "Bluebaum,M", "Zhigalko,S", 
                      "Vlassov,N", "Andreikin,D","Rustemov,A"}

# Rango de archivos TWIC a procesar (inclusive) - Hemos elegido DICIEMBRE 2024 - MAYO 2025 (6 meses)
numero_minimo = 1569
numero_maximo = 1592

## 2. Selección y guardado de las partidas de los jugadores objetivo

In [3]:
## Partidas 960 y con errores son descartadas

patron_twic = re.compile(r"twic(\d+)\.pgn", re.IGNORECASE)

# Rutas HDFS
ruta_entrada_hdfs = "/user/ajedrez/raw"
carpeta_salida_hdfs = "/user/ajedrez/partidas"
archivo_salida_hdfs = f"{carpeta_salida_hdfs}/partidas_grupo.pgn"

# Crear carpeta en HDFS si no existe
subprocess.run(["hdfs", "dfs", "-mkdir", "-p", carpeta_salida_hdfs])

# Ruta temporal local
ruta_temp_local = Path("./partidas_grupo_tmp.pgn")

# Filtrado y escritura temporal en local
partidas_guardadas = set()
contador = 0

# Obtener lista de archivos PGN en HDFS
resultado_ls = subprocess.run(
    ["hdfs", "dfs", "-ls", ruta_entrada_hdfs],
    capture_output=True, text=True
)

lineas = resultado_ls.stdout.splitlines()
archivos_pgn = [
    linea.split()[-1] for linea in lineas if linea.endswith(".pgn")
]

with open(ruta_temp_local, "w", encoding="utf-8") as salida:
    for ruta in archivos_pgn:
        nombre = Path(ruta).name
        match = patron_twic.match(nombre)
        if match:
            numero = int(match.group(1))
            if not (numero_minimo <= numero <= numero_maximo):
                continue

        print(f"Leyendo {nombre} desde HDFS")

        resultado_cat = subprocess.run(
            ["hdfs", "dfs", "-cat", ruta],
            capture_output=True, text=False  # leer como binario
        )
        contenido = resultado_cat.stdout.decode("utf-8", errors="ignore") 
        buffer = io.StringIO(contenido)

        while True:
            try:
                partida = chess.pgn.read_game(buffer)
                if partida is None:
                    break
            except Exception as e:
                print(f" Error al leer partida: {e}")
                continue

            if partida.headers.get("Variant", "").lower() == "chess960":
                continue

            blanco = partida.headers.get("White", "Unknown")
            negro = partida.headers.get("Black", "Unknown")

            if blanco in jugadores_objetivo or negro in jugadores_objetivo:
                clave = tuple(sorted([blanco, negro])) + (partida.headers.get("Result", ""),)

                if clave not in partidas_guardadas:
                    try:
                        tablero = partida.board()
                        for movimiento in partida.mainline_moves():
                            if movimiento not in tablero.legal_moves:
                                raise ValueError(f"Movimiento ilegal: {tablero.san(movimiento)}")
                            tablero.push(movimiento)

                        texto_partida = str(partida)
                    except Exception as e:
                        print(f" Partida inválida {blanco} vs {negro}: {e}")
                        continue

                    partidas_guardadas.add(clave)
                    salida.write(texto_partida + "\n\n")
                    contador += 1

print(f"\n {contador} partidas seleccionadas. Subiendo ahora a HDFS:")

# Subir a HDFS (sobrescribe si ya existe)
subprocess.run(["hdfs", "dfs", "-put", "-f", str(ruta_temp_local), archivo_salida_hdfs])

# Eliminar temporal
ruta_temp_local.unlink()

print(f"Archivo guardado en HDFS: {archivo_salida_hdfs}")

Leyendo twic1569.pgn desde HDFS
Leyendo twic1570.pgn desde HDFS
Leyendo twic1571.pgn desde HDFS
Leyendo twic1572.pgn desde HDFS
Leyendo twic1573.pgn desde HDFS
Leyendo twic1574.pgn desde HDFS
Leyendo twic1575.pgn desde HDFS
Leyendo twic1576.pgn desde HDFS
Leyendo twic1577.pgn desde HDFS
Leyendo twic1578.pgn desde HDFS
Leyendo twic1579.pgn desde HDFS
Leyendo twic1580.pgn desde HDFS
Leyendo twic1581.pgn desde HDFS
Leyendo twic1582.pgn desde HDFS
Leyendo twic1583.pgn desde HDFS
Leyendo twic1584.pgn desde HDFS
Leyendo twic1585.pgn desde HDFS


illegal san: 'Nxe4' in rnbqkbnr/pppppppp/8/8/8/5P2/PPPPP1PP/RNBQKBNR b KQkq - 0 1 while parsing <Game at 0x763298fdc4f0 ('Antal,Ge' vs. 'Predke,A', '2025.02.23' at 'Germany GER')>


Leyendo twic1586.pgn desde HDFS
Leyendo twic1587.pgn desde HDFS
Leyendo twic1588.pgn desde HDFS
Leyendo twic1589.pgn desde HDFS
Leyendo twic1590.pgn desde HDFS
Leyendo twic1591.pgn desde HDFS
Leyendo twic1592.pgn desde HDFS

 3013 partidas seleccionadas. Subiendo ahora a HDFS:
Archivo guardado en HDFS: /user/ajedrez/partidas/partidas_grupo.pgn


## 3. División del archivo de partidas en lotes

In [3]:
def dividir_pgn_en_lotes(pgn_path, partidas_por_lote, carpeta_salida="lotes_pgn"):
    os.makedirs(carpeta_salida, exist_ok=True)
    base_nombre = os.path.splitext(os.path.basename(pgn_path))[0]

    with open(pgn_path, encoding="utf-8", errors="ignore") as archivo_pgn:
        contador_partidas = 0
        contador_lote = 1
        archivo_salida = None

        while True:
            partida = chess.pgn.read_game(archivo_pgn)
            if partida is None:
                break

            if contador_partidas % partidas_por_lote == 0:
                if archivo_salida:
                    archivo_salida.close()
                nombre_archivo_lote = f"{base_nombre}_{contador_lote:03}.pgn"
                ruta_lote = os.path.join(carpeta_salida, nombre_archivo_lote)
                archivo_salida = open(ruta_lote, "w", encoding="utf-8")
                contador_lote += 1

            archivo_salida.write(str(partida) + "\n\n")
            contador_partidas += 1

        if archivo_salida:
            archivo_salida.close()

    print(f" Se dividió '{pgn_path}' en {contador_lote - 1} archivos en '{carpeta_salida}'")
    return carpeta_salida

In [4]:
# Descargar archivo desde HDFS a local
# No se puede usar hdfs dfs -cat directamente en un bucle y además chess.pgn.read_game() necesita un archivo secuencial
# quizás se puede hacer con hdfs3

ruta_hdfs = "/user/ajedrez/partidas/partidas_grupo.pgn"
ruta_local = Path("./partidas_grupo.pgn")
subprocess.run(["hdfs", "dfs", "-get", "-f", ruta_hdfs, str(ruta_local)])

# Carpeta local donde se guardan los lotes
carpeta_lotes_local = Path("./lotes_pgn")
carpeta_lotes_local.mkdir(exist_ok=True)

# Dividir archivo local en lotes de 250 partidas cada uno
dividir_pgn_en_lotes(str(ruta_local), partidas_por_lote=250, carpeta_salida=str(carpeta_lotes_local))

# Subir lotes a HDFS
carpeta_hdfs_destino = "/user/ajedrez/partidas/lotes_pgn"
subprocess.run(["hdfs", "dfs", "-mkdir", "-p", carpeta_hdfs_destino])
subprocess.run(["hdfs", "dfs", "-put", "-f"] + [str(f) for f in carpeta_lotes_local.glob("*.pgn")] + [carpeta_hdfs_destino])

print(f" Todos los lotes subidos a HDFS en: {carpeta_hdfs_destino}")

 Se dividió 'partidas_grupo.pgn' en 13 archivos en 'lotes_pgn'
 Todos los lotes subidos a HDFS en: /user/ajedrez/partidas/lotes_pgn


## 4. Análisis de las partidas

Para cada partida de un fichero PGN generaremos un diccionario que recopilará las características de cada jugador. Estas características se volcarán en un dataframe que quedará guardado como un archivo parquet. Los datos recopilados en cada partida vendrán dados por dos filas (una para cada jugador) y 33 columnas, que son:

---
### Identificación del jugador
- **`jugador`**: Nombre del jugador (ej. `"Carlsen,M"`).
- **`color`**: Color con el que jugó el jugador: `"W"` (blancas) o `"B"` (negras).
- **`fide_id`**: ID FIDE del jugador (único para cada jugador).
---
###  Información contextual
- **`evento`**: Nombre del torneo o evento.
- **`lugar`**: Lugar donde se jugó la partida.
- **`fechas`**: Fecha convertida al tipo `datetime` (útil para orden y filtrado).
---
### Rendimiento y resultado
- **`elo`**: Rating Elo del jugador en el momento de la partida (según color).
- **`resultados`**: Resultado desde la perspectiva del jugador: (`1.0` → victoria, `0.5` → empate, `0.0` → derrota)
---
### Evaluaciones del motor Stockfish (en centipawns)
- **`evaluacion_cp`**: Promedio de evaluación durante toda la partida.
- **`evaluacion_cp_apertura`**: Promedio de evaluación en la apertura (movimientos 1–20).
- **`evaluacion_cp_mediojuego`**: Promedio en el mediojuego (mov. 21–60).
- **`evaluacion_cp_final`**: Promedio en el final (mov. 61 en adelante).
---
### Conteo de errores según pérdida de centipawns
- **`errores_graves`**: Blunders (pérdida > 300 cp). Desglosados en:
    - *`errores_graves_apertura`*, *`errores_graves_mediojuego`*, *`errores_graves_final`*
- **`errores`**: Errores (pérdida entre 100 y 300 cp). Desglosados en:
    - *`errores_apertura`*, *`errores_mediojuego`*, *`errores_final`*
- **`errores_leves`**: Imprecisiones (pérdida entre 50 y 100 cp). Desglosados en:
    - *`errores_leves_apertura`*, *`errores_leves_mediojuego`*, *`errores_leves_final`* 
---
### Estadísticas del estilo de juego
- **`movimientos_total`**: Número total de jugadas realizadas en la partida.
- **`mov_peones`**: Jugadas realizadas con peones.
- **`mov_centrales`**: Jugadas a casillas centrales (`d4`, `e4`, `d5`, `e5`).
- **`intercambio_piezas`**: Veces que el jugador capturó una pieza.
- **`sacrificios`**: Sacrificios (captura de piezas de menor valor que la propia con ventaja en la evaluación).
- **`enroque`**: Movimiento en que se enrocó (o 0 si no se enrocó).
---
### Información adicional
- **`cod_eco`**: Código ECO de apertura (ej. `"B90"`).
- **`pgn`**: Texto completo de la partida en formato PGN.
- **`san`**: Lista de jugadas en notación algebraica estándar (SAN).


In [4]:
def analizar_pgn(pgn_path: str, stockfish_path: str = "stockfish.exe"):
    log_path = "errores_pgn.log"
    central_squares = {"d4", "e4", "d5", "e5"}
    
    piece_value = {"p": 1, "n": 3, "b": 3, "r": 5, "q": 9, "k": 0}
    stockfish = Stockfish(path=stockfish_path)

    players = defaultdict(lambda: {
        "fide_id": None,
        "elo": [],
        "resultados": [],
        "partidas": 0,
        "movimientos_total": [],
        "enroque": [],
        "intercambio_piezas": [],
        "sacrificios": [],
        "mov_peones": [],
        "mov_centrales": [],
        "cod_eco": [],
        "evaluacion_cp": [],
        "evaluacion_cp_apertura": [],
        "evaluacion_cp_mediojuego": [],
        "evaluacion_cp_final": [],
        "errores_graves": [],
        "errores": [],
        "errores_leves": [],
        "errores_graves_apertura": [],
        "errores_graves_mediojuego": [],
        "errores_graves_final": [],
        "errores_apertura": [],
        "errores_mediojuego": [],
        "errores_final": [],
        "errores_leves_apertura": [],
        "errores_leves_mediojuego": [],
        "errores_leves_final": [],
        "evento": [],
        "lugar": [],
        "pgn": [],
        "san": [],
        "fechas": []
    })

    jugadores_objetivo = set()
    with open(pgn_path, encoding="utf-8", errors="ignore") as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break
            jugadores_objetivo.add(game.headers.get("White", "Unknown"))
            jugadores_objetivo.add(game.headers.get("Black", "Unknown"))
    jugadores_objetivo = {j for j in jugadores_objetivo if j not in {"?", "Unknown", ""}}

    with open(pgn_path, encoding="utf-8", errors="ignore") as f:
        total_games = sum(1 for _ in iter(lambda: chess.pgn.read_game(f), None))

    with open(log_path, "w", encoding="utf-8") as log_file:
        log_file.write(f"Registro de errores - {datetime.datetime.now()}\n\n")

    with open(pgn_path, encoding="utf-8", errors="ignore") as f:
        for i in tqdm(range(total_games), desc="Analizando partidas"):
            try:
                game = chess.pgn.read_game(f)
                if game is None:
                    break

                white = game.headers.get("White", "Unknown")
                black = game.headers.get("Black", "Unknown")
                white_fide = game.headers.get("WhiteFideId", "None")
                black_fide = game.headers.get("BlackFideId", "None")
                white_elo = int(game.headers.get("WhiteElo", "0"))
                black_elo = int(game.headers.get("BlackElo", "0"))
                resultado_raw = game.headers.get("Result", "*")

                exporter = chess.pgn.StringExporter(headers=True, variations=False, comments=False)
                pgn_str = game.accept(exporter)

                board = game.board()
                jugadas_san = []
                for move in game.mainline_moves():
                    jugadas_san.append(board.san(move))
                    board.push(move)

                resultado_white, resultado_black = {
                    "1-0": (1.0, 0.0),
                    "0-1": (0.0, 1.0),
                    "1/2-1/2": (0.5, 0.5)
                }.get(resultado_raw, (None, None))

                if white not in jugadores_objetivo and black not in jugadores_objetivo:
                    continue

                eco = game.headers.get("ECO", "None")
                fecha_str = game.headers.get("Date", None)
                try:
                    fecha = datetime.datetime.strptime(fecha_str, "%Y.%m.%d").date()
                except:
                    fecha = None

                game_counted = {"W": False, "B": False}
                board = game.board()
                move_number = 0
                moves_list = []
                jugador_stats_temporales = {"W": defaultdict(int), "B": defaultdict(int)}
                contador_jugadas_por_jugador = {"W": 0, "B": 0}
                evaluaciones_por_fase = {"apertura": [], "mediojuego": [], "final": [], "total": []}

                for move in game.mainline_moves():
                    move_number += 1
                    current_player = white if board.turn else black
                    color_tag = "W" if board.turn else "B"

                    if current_player not in jugadores_objetivo:
                        board.push(move)
                        continue

                    contador_jugadas_por_jugador[color_tag] += 1

                    key = (current_player, color_tag)
                    stats = players[key]
                    stats_temp = jugador_stats_temporales[color_tag]

                    if not game_counted[color_tag]:
                        stats["partidas"] += 1
                        stats["cod_eco"].append(eco)
                        stats["fide_id"] = white_fide if color_tag == "W" else black_fide
                        stats["elo"].append(white_elo if color_tag == "W" else black_elo)
                        stats["resultados"].append(resultado_white if color_tag == "W" else resultado_black)
                        if fecha:
                            stats["fechas"].append(fecha)
                        stats["evento"].append(game.headers.get("Event", ""))
                        stats["lugar"].append(game.headers.get("Site", ""))
                        stats["pgn"].append(pgn_str)
                        stats["san"].append(jugadas_san)

                    game_counted[color_tag] = True
                    uci = move.uci()
                    moves_list.append(uci)

                    try:
                        stockfish.set_position(moves_list[:-1])
                        eval_before = stockfish.get_evaluation()["value"] or 0
                        stockfish.set_position(moves_list)
                        eval_after = stockfish.get_evaluation()["value"] or 0
                    except Exception as sf_error:
                        raise RuntimeError(f"Stockfish error en jugada {uci}: {sf_error}")

                    loss = abs(eval_before - eval_after)
                    fase = "apertura" if move_number <= 20 else "mediojuego" if move_number <= 60 else "final"
                    eval_abs = abs(eval_after)
                    evaluaciones_por_fase[fase].append(eval_abs)
                    evaluaciones_por_fase["total"].append(eval_abs)

                    piece = board.piece_at(move.from_square)
                    if not board.is_legal(move):
                        raise ValueError(f"Movimiento ilegal: {uci} en {board.fen()}")

                    san = board.san(move)
                    board.push(move)

                    stats_temp["movimientos_total"] += 1

                    if san in ["O-O", "O-O-O"] and "enroque" not in stats_temp:
                        stats_temp["enroque"] = contador_jugadas_por_jugador[color_tag]

                    if "x" in san:
                        stats_temp["intercambio_piezas"] += 1
                        capturada = board.piece_at(move.to_square)
                        if capturada:
                            atacante = piece_value[piece.symbol().lower()]
                            capturada_valor = piece_value[capturada.symbol().lower()]
                            if atacante > capturada_valor and loss < 150:
                                stats_temp["sacrificios"] += 1

                    if piece and piece.symbol().lower() == "p":
                        stats_temp["mov_peones"] += 1

                    if chess.square_name(move.to_square) in central_squares:
                        stats_temp["mov_centrales"] += 1

                    if loss > 300:
                        stats_temp["errores_graves"] += 1
                        stats_temp[f"blunders_{fase}"] += 1
                    elif loss > 100:
                        stats_temp["errores"] += 1
                        stats_temp[f"errores_{fase}"] += 1
                    elif loss > 50:
                        stats_temp["errores_leves"] += 1
                        stats_temp[f"imprecisiones_{fase}"] += 1

                for color_tag, jugador in [("W", white), ("B", black)]:
                    if jugador in jugadores_objetivo:
                        stats = players[(jugador, color_tag)]
                        temp = jugador_stats_temporales[color_tag]
                        stats["movimientos_total"].append(temp["movimientos_total"])
                        stats["mov_peones"].append(temp["mov_peones"])
                        stats["mov_centrales"].append(temp["mov_centrales"])
                        stats["intercambio_piezas"].append(temp["intercambio_piezas"])
                        stats["sacrificios"].append(temp["sacrificios"])
                        stats["enroque"].append(temp["enroque"] if "enroque" in temp else 0)

                        for fase_eval in ["total", "apertura", "mediojuego", "final"]:
                            lista = evaluaciones_por_fase[fase_eval]
                            campo = f"evaluacion_cp" if fase_eval == "total" else f"evaluacion_cp_{fase_eval}"
                            stats[campo].append(statistics.mean(lista) if lista else None)

                        errores_map = {
                            "errores_graves": "errores_graves",
                            "errores": "errores",
                            "errores_leves": "errores_leves",
                            "errores_graves_apertura": "blunders_apertura",
                            "errores_graves_mediojuego": "blunders_mediojuego",
                            "errores_graves_final": "blunders_final",
                            "errores_apertura": "errores_apertura",
                            "errores_mediojuego": "errores_mediojuego",
                            "errores_final": "errores_final",
                            "errores_leves_apertura": "imprecisiones_apertura",
                            "errores_leves_mediojuego": "imprecisiones_mediojuego",
                            "errores_leves_final": "imprecisiones_final"
                        }

                        for campo_destino, campo_temp in errores_map.items():
                            valor = temp.get(campo_temp, 0)
                            stats[campo_destino].append(valor)

            except Exception as e:
                msg = f" Error en partida #{i+1} ({white} vs {black}): {e}"
                print("\n" + msg)
                with open(log_path, "a", encoding="utf-8") as log_file:
                    log_file.write(msg + "\n")

    filas = []
    for (nombre, color), datos in players.items():
        for i in range(datos["partidas"]):
            fila = {"jugador": nombre, "color": color, "fide_id": datos.get("fide_id", "")}
            for campo, valores in datos.items():
                if isinstance(valores, list) and len(valores) > i:
                    fila[campo] = valores[i]
            filas.append(fila)

    df_partidas = pd.DataFrame(filas)
    df_partidas["fechas"] = pd.to_datetime(df_partidas["fechas"], errors="coerce")
    df_partidas = df_partidas.fillna(np.nan)

    nombre_archivo = f"analisis_{os.path.splitext(os.path.basename(pgn_path))[0]}.parquet"
    df_partidas.to_parquet(nombre_archivo, engine="pyarrow", coerce_timestamps="ms")
    return nombre_archivo

## 5. Llamada a la función de análisis y guardado en HFDS (.parquet)

In [ ]:
# Directorios
carpeta_hdfs_lotes = "/user/ajedrez/partidas/lotes_pgn"
carpeta_hdfs_analizados = "/user/ajedrez/partidas/analizadas_grupo"
carpeta_local_temporal = Path("./lotes_tmp")
carpeta_local_temporal.mkdir(exist_ok=True)

# Crear carpeta en HDFS de destino si no existe
subprocess.run(["hdfs", "dfs", "-mkdir", "-p", carpeta_hdfs_analizados])

#  Listar los archivos .pgn en HDFS
resultado_ls = subprocess.run(
    ["hdfs", "dfs", "-ls", carpeta_hdfs_lotes],
    capture_output=True, text=True
)
archivos_hdfs = [
    linea.split()[-1] for linea in resultado_ls.stdout.splitlines()
    if linea.strip().endswith(".pgn")
]

for ruta_hdfs in archivos_hdfs:
    nombre_archivo = Path(ruta_hdfs).name
    ruta_local = carpeta_local_temporal / nombre_archivo

    print(f"Descargando {nombre_archivo} desde HDFS...")
    subprocess.run(["hdfs", "dfs", "-get", "-f", ruta_hdfs, str(ruta_local)])

    try:
        parquet_local = analizar_pgn(str(ruta_local), stockfish_path="stockfish")

        # Subir a HDFS el resultado en .parquet
        subprocess.run(["hdfs", "dfs", "-put", "-f", parquet_local, carpeta_hdfs_analizados])
        print(f"Subido {parquet_local} → HDFS")

        # Limpiar
        os.remove(ruta_local)
        os.remove(parquet_local)

    except Exception as e:
        print(f"Error al analizar {nombre_archivo}: {e}")
